In [10]:
import pandas as pd
import Funciones as f
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle

In [7]:
%%capture
%run "43. Exportar.ipynb"

In [12]:
def Crear_Tabla_Compacta_Gen_Ball(
    Diccionario_Resultados,
    Diccionario_P_Valores,
    Categoria_A_Mostrar = 'Left_Wing',
    Items_A_Incluir = None,
    Nombre_Archivo = 'Tabla_LW_Gen_Ball.png',
    Carpeta_Destino = 'Tablas_Estadisticas'
):

    """
    Crea una tabla compacta mostrando Generales y Ballotage 
    para una categoría específica.
    Formato: Media±DE (Izq/Der) | n | Significancia
    
    """

    import os
    if not os.path.exists(Carpeta_Destino):
        os.makedirs(Carpeta_Destino)

    # Etiqueta corta para la categoría.
    Etiquetas_Cortas = {
        'Left_Wing': 'Left Wing',
        'Progressivism': 'Progressivism',
        'Centre': 'Centre',
        'Moderate_Right_A': 'Moderate Right A',
        'Moderate_Right_B': 'Moderate Right B',
        'Right_Wing_Libertarian': 'Right Wing Libertarian'
    }
    
    Etiqueta_Categoria = Etiquetas_Cortas.get(
        Categoria_A_Mostrar, 
        Categoria_A_Mostrar
    )

    # Obtener números de ítems.
    if Items_A_Incluir is None:
        Items_Numeros = set()
        for Dataset in ['Generales', 'Ballotage']:
            if Dataset in Diccionario_Resultados:
                if Categoria_A_Mostrar in Diccionario_Resultados[Dataset]:
                    for Item in Diccionario_Resultados[Dataset][Categoria_A_Mostrar].keys():
                        if 'CO_Item_' in Item:
                            Partes = Item.split('_')
                            if len(Partes) >= 3 and Partes[2].isdigit():
                                Items_Numeros.add(int(Partes[2]))
        Items_Numeros = sorted(list(Items_Numeros))[:10]  # Limitar a 10 ítems.
    else:
        Items_Numeros = Items_A_Incluir

    # Preparar datos para la tabla.
    Datos_Tabla = []
    
    # Encabezado principal con el nombre de la categoría.
    Fila_Categoria = ['', f'{Etiqueta_Categoria}', '']
    Datos_Tabla.append(Fila_Categoria)
    
    # Sub-encabezado con Generales y Ballotage.
    Fila_Datasets = ['', 'Generales', 'Ballotage']
    Datos_Tabla.append(Fila_Datasets)
    
    # Línea separadora.
    Datos_Tabla.append(['—' * 15, '—' * 30, '—' * 30])
    
    for Num_Item in Items_Numeros:
        # Variables a buscar.
        Var_Izq = f'CO_Item_{Num_Item}_Izq'
        Var_Der = f'CO_Item_{Num_Item}_Der'
        
        # Título del ítem.
        Fila_Item_Titulo = [f'Ítem {Num_Item}', '', '']
        Datos_Tabla.append(Fila_Item_Titulo)
        
        # Preparar datos para Generales y Ballotage.
        Datos_Media = ['  Media±DE (Izq/Der)']
        Datos_N = ['  n (Izq/Der)']
        Datos_Sig = ['  Significancia']
        
        for Dataset in ['Generales', 'Ballotage']:
            # Buscar estadísticas.
            Stats_Izq = None
            Stats_Der = None
            P_Valor = None
            
            if Dataset in Diccionario_Resultados:
                if Categoria_A_Mostrar in Diccionario_Resultados[Dataset]:
                    if Var_Izq in Diccionario_Resultados[Dataset][Categoria_A_Mostrar]:
                        Stats_Izq = Diccionario_Resultados[Dataset][Categoria_A_Mostrar][Var_Izq]
                    if Var_Der in Diccionario_Resultados[Dataset][Categoria_A_Mostrar]:
                        Stats_Der = Diccionario_Resultados[Dataset][Categoria_A_Mostrar][Var_Der]
            
            # Buscar p-valor.
            if Dataset in Diccionario_P_Valores:
                if Var_Izq in Diccionario_P_Valores[Dataset]:
                    P_Valor = Diccionario_P_Valores[Dataset][Var_Izq]
            
            # Formatear Media±DE (Izq/Der).
            Texto_Media = '—'
            Texto_N = '—'
            Texto_Sig = '—'
            
            if Stats_Izq and Stats_Der:
                Media_Izq = Stats_Izq.get('Media', np.nan)
                DE_Izq = Stats_Izq.get('Desvio_Estandar', np.nan)
                N_Izq = Stats_Izq.get('N', 0)
                
                Media_Der = Stats_Der.get('Media', np.nan)
                DE_Der = Stats_Der.get('Desvio_Estandar', np.nan)
                N_Der = Stats_Der.get('N', 0)
                
                if not np.isnan(Media_Izq) and not np.isnan(Media_Der):
                    Texto_Media = f'{Media_Izq:.2f}±{DE_Izq:.2f} / {Media_Der:.2f}±{DE_Der:.2f}'
                    Texto_N = f'{N_Izq} / {N_Der}'
                    
                    # Determinar significancia.
                    if P_Valor is not None and not np.isnan(P_Valor):
                        if P_Valor < 0.001:
                            Texto_Sig = '***'
                        elif P_Valor < 0.01:
                            Texto_Sig = '**'
                        elif P_Valor < 0.05:
                            Texto_Sig = '*'
                        else:
                            Texto_Sig = 'ns'
            
            Datos_Media.append(Texto_Media)
            Datos_N.append(Texto_N)
            Datos_Sig.append(Texto_Sig)
        
        # Agregar las 3 filas del ítem.
        Datos_Tabla.append(Datos_Media)
        Datos_Tabla.append(Datos_N)
        Datos_Tabla.append(Datos_Sig)
        
        # Línea separadora entre ítems.
        Datos_Tabla.append(['', '', ''])
    
    # Crear figura.
    Num_Filas = len(Datos_Tabla)
    
    Fig = plt.figure(figsize=(12, Num_Filas * 0.35))
    Ax = Fig.add_subplot(111)
    Ax.axis('tight')
    Ax.axis('off')
    
    # Crear tabla sin encabezados (ya están en los datos).
    Tabla = Ax.table(
        cellText = Datos_Tabla,
        cellLoc = 'left',
        loc = 'center',
        colWidths = [0.25, 0.375, 0.375]
    )
    
    # Estilizar.
    Tabla.auto_set_font_size(False)
    Tabla.set_fontsize(9)
    Tabla.scale(1.0, 1.3)
    
    # Colorear filas según contenido.
    for i in range(1, Num_Filas + 1):
        if i <= len(Datos_Tabla):
            Primera_Columna = str(Datos_Tabla[i-1][0])
            
            # Fila de categoría (título principal).
            if i == 1:
                for j in range(3):
                    Tabla[(i, j)].set_facecolor('#2C5282')
                    Tabla[(i, j)].set_text_props(
                        weight = 'bold',
                        color = 'white',
                        size = 11
                    )
                # Fusionar visualmente las celdas.
                Tabla[(i, 0)].set_text_props(ha = 'center')
            
            # Fila de Generales/Ballotage.
            elif i == 2:
                for j in range(3):
                    Tabla[(i, j)].set_facecolor('#5B9BD5')
                    Tabla[(i, j)].set_text_props(
                        weight = 'bold',
                        color = 'white'
                    )
            
            # Líneas separadoras.
            elif '—' in Primera_Columna:
                for j in range(3):
                    Tabla[(i, j)].set_facecolor('#F2F2F2')
                    Tabla[(i, j)].set_height(0.01)
            
            # Títulos de ítem.
            elif 'Ítem' in Primera_Columna and not 'Media' in Primera_Columna:
                for j in range(3):
                    Tabla[(i, j)].set_facecolor('#D9E2F3')
                    if j == 0:
                        Tabla[(i, j)].set_text_props(
                            weight = 'bold',
                            size = 10
                        )
            
            # Filas de Media.
            elif 'Media' in Primera_Columna:
                Tabla[(i, 0)].set_facecolor('#E7E6E6')
                Tabla[(i, 0)].set_text_props(style = 'italic')
                for j in [1, 2]:
                    Tabla[(i, j)].set_facecolor('white')
                    # Resaltar valores en negrita.
                    Tabla[(i, j)].set_text_props(weight = 'semibold')
            
            # Filas de n.
            elif 'n (' in Primera_Columna:
                Tabla[(i, 0)].set_facecolor('#E7E6E6')
                Tabla[(i, 0)].set_text_props(style = 'italic')
                for j in [1, 2]:
                    Tabla[(i, j)].set_facecolor('#F8F8F8')
            
            # Filas de Significancia.
            elif 'Significancia' in Primera_Columna:
                Tabla[(i, 0)].set_facecolor('#E7E6E6')
                Tabla[(i, 0)].set_text_props(style = 'italic')
                
                for j in [1, 2]:
                    if j < len(Datos_Tabla[i-1]):
                        Texto_Sig = str(Datos_Tabla[i-1][j])
                        
                        # Colorear según nivel de significancia.
                        if '***' in Texto_Sig:
                            Tabla[(i, j)].set_facecolor('#90EE90')
                            Tabla[(i, j)].set_text_props(
                                weight = 'bold',
                                color = '#006400'
                            )
                        elif '**' in Texto_Sig:
                            Tabla[(i, j)].set_facecolor('#B8F4B8')
                            Tabla[(i, j)].set_text_props(
                                weight = 'bold',
                                color = '#228B22'
                            )
                        elif '*' in Texto_Sig:
                            Tabla[(i, j)].set_facecolor('#D4F4D4')
                            Tabla[(i, j)].set_text_props(weight = 'bold')
                        elif 'ns' in Texto_Sig:
                            Tabla[(i, j)].set_facecolor('#FFE4E1')
                            Tabla[(i, j)].set_text_props(color = '#8B0000')
                        else:
                            Tabla[(i, j)].set_facecolor('white')
    
    # Título principal.
    Titulo_Principal = (
        f'Comparación de Cambios de Opinión: {Etiqueta_Categoria}\n'
        'Generales vs Ballotage - Respuestas a ítems asociados con candidatos Izq/Der'
    )
    
    Fig.suptitle(
        Titulo_Principal,
        fontsize = 13,
        fontweight = 'bold',
        y = 0.98
    )
    
    # Leyenda.
    Texto_Leyenda = (
        'Significancia: *** p<0.001 | ** p<0.01 | * p<0.05 | ns: no significativo\n'
        'Formato: Media±DE (Izquierda / Derecha) | n (Izquierda / Derecha)'
    )
    
    Fig.text(
        0.5, 0.02,
        Texto_Leyenda,
        ha = 'center',
        fontsize = 9,
        style = 'italic'
    )
    
    # Guardar.
    Ruta_Completa = os.path.join(
        Carpeta_Destino,
        Nombre_Archivo
    )
    
    Fig.savefig(
        Ruta_Completa,
        dpi = 300,
        bbox_inches = 'tight',
        facecolor = 'white'
    )
    
    plt.close(Fig)
    
    print(f"✅ Tabla guardada en: {Ruta_Completa}")
    
    return Ruta_Completa


In [14]:
Items_Ejemplo = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]

Ruta_Tabla_LW = Crear_Tabla_Compacta_Gen_Ball(
    Diccionario_Resultados = Diccionario_Resultados_CO_Individuales,
    Diccionario_P_Valores = Diccionario_P_Valores,
    Categoria_A_Mostrar = 'Left_Wing',
    Items_A_Incluir = Items_Ejemplo,
    Nombre_Archivo = 'Tabla_Left_Wing_Gen_Ball.png',
    Carpeta_Destino = 'Tablas_Estadisticas'
)

print(f"\n📊 Tabla para Left Wing creada")
print(f"   Comparación Generales vs Ballotage")
print(f"   Ítems incluidos: {Items_Ejemplo}")

✅ Tabla guardada en: Tablas_Estadisticas\Tabla_Left_Wing_Gen_Ball.png

📊 Tabla para Left Wing creada
   Comparación Generales vs Ballotage
   Ítems incluidos: [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
